In [2]:
!pip install keras

In [5]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [25]:
import pandas as pd
import numpy as np
dt = pd.read_csv('../train.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
dt = dt.drop("CUS_ID" ,axis=1)

ohi = pd.get_dummies(dt["GENDER"])
ohi
dt = dt.drop("GENDER",axis=1)
dt = pd.concat([ohi,dt],axis=1)

sizemap = {
    "低":1,
    "中":2,
    "中高":3,
    "高":4
}

dt["AGE"] = dt["AGE"].map(sizemap)

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in dt.keys():    
    for a in dt[i]:
        if  a is not None and isinstance(a,str):
            dt[i] = labelencoder.fit_transform(dt[i].astype(str))
            break
            
for i in dt.keys():
    dt[i] = dt[i].fillna(dt[i].mean())

In [27]:
dt = dt.sample(frac = 1)

fraud_dt = dt.loc[dt['Y1'] == 1]
non_fraud_dt = dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dt = normal_distributed_dt.sample(frac=1, random_state=3)

new_dt.shape

(4000, 132)

In [34]:
dt.head()

,F,M,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,...,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
25215,0,1,1,2,2,1.000000,0.000000,0,0,0,...,0.00000,2,2,2,2,2,2,2,2,0
48615,1,0,2,2,2,2.169075,0.315289,0,0,0,...,0.00000,5,2,2,2,2,2,2,2,1
40559,1,0,1,2,3,2.000000,0.000000,0,0,0,...,0.00000,2,1,0,0,0,0,0,0,0
85893,0,1,3,0,0,3.000000,0.000000,1,0,0,...,0.00213,4,2,2,2,2,2,2,2,0
56030,1,0,1,2,3,2.000000,0.000000,0,0,0,...,0.00000,2,2,2,2,2,2,2,2,0


In [28]:
X = new_dt.drop("Y1",axis=1)
y = new_dt['Y1']

In [29]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
n_inputs = X_train.shape[1]

undersample_model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs, ), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

In [31]:
undersample_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 131)               17292     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                4224      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 66        
Total params: 21,582
Trainable params: 21,582
Non-trainable params: 0
_________________________________________________________________


In [32]:
undersample_model.compile(Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
x1 = dt.drop("Y1",axis=1)
y1 = dt["Y1"]

In [41]:
undersample_model.fit(x1,y1, validation_split=0.3, batch_size=20, epochs=20, shuffle=True, verbose=2)

Train on 70000 samples, validate on 30000 samples
Epoch 1/20
 - 4s - loss: 0.0718 - accuracy: 0.9805 - val_loss: 0.0954 - val_accuracy: 0.9770
Epoch 2/20
 - 4s - loss: 0.0707 - accuracy: 0.9808 - val_loss: 0.0979 - val_accuracy: 0.9794
Epoch 3/20
 - 4s - loss: 0.0700 - accuracy: 0.9809 - val_loss: 0.0948 - val_accuracy: 0.9791
Epoch 4/20
 - 4s - loss: 0.0694 - accuracy: 0.9810 - val_loss: 0.1037 - val_accuracy: 0.9795
Epoch 5/20
 - 4s - loss: 0.0690 - accuracy: 0.9808 - val_loss: 0.1060 - val_accuracy: 0.9797
Epoch 6/20
 - 4s - loss: 0.0681 - accuracy: 0.9813 - val_loss: 0.0999 - val_accuracy: 0.9788
Epoch 7/20
 - 4s - loss: 0.0676 - accuracy: 0.9813 - val_loss: 0.0997 - val_accuracy: 0.9782
Epoch 8/20
 - 4s - loss: 0.0669 - accuracy: 0.9813 - val_loss: 0.1008 - val_accuracy: 0.9793
Epoch 9/20
 - 4s - loss: 0.0658 - accuracy: 0.9816 - val_loss: 0.1038 - val_accuracy: 0.9789
Epoch 10/20
 - 4s - loss: 0.0654 - accuracy: 0.9816 - val_loss: 0.1040 - val_accuracy: 0.9769
Epoch 11/20
 - 4s -

In [39]:

undersample_model.val

AttributeError: 'Sequential' object has no attribute 'val'

In [16]:
import pandas as pd
from sklearn import ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

new_dtf = pd.read_csv('../train.csv' , encoding='big5')
xf = new_dtf.drop("Y1",axis=1)
yf = new_dtf["Y1"]
train_x, test_x, train_y, test_y = train_test_split(xf, yf, test_size = 0.3)
 
forest = ensemble.RandomForestClassifier(n_estimators = 131)
forest_fit = forest.fit(train_x, train_y)

test_y_predicted = forest.predict(test_x)
 
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)


SyntaxError: invalid character in identifier (<ipython-input-16-5338fcbf4cc4>, line 9)

In [14]:
 
# forest = ensemble.RandomForestClassifier(n_estimators = 131)
# forest_fit = forest.fit(train_x, train_y)

# test_y_predicted = forest.predict(test_x)
 
# fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
# auc = metrics.auc(fpr, tpr)
# print(auc)

SyntaxError: invalid character in identifier (<ipython-input-14-bcaa31d44e43>, line 5)

SyntaxError: invalid character in identifier (<ipython-input-15-bcaa31d44e43>, line 5)